# Aula 2 - Reinforcement Learning

## Tutorial: Q Learning no ambiente FrozenLake

### Prof. Dr. Ahirton Lopes (profahirton.lopes@fiap.com.br)

# Q* Learning com FrozenLake 4x4

Neste Notebook, implementaremos um agente <b>que reproduz o desafio FrozenLake.</b>

![texto alternativo](https://www.gymlibrary.dev/_images/frozen_lake.gif)

O objetivo deste jogo é <b>passar do estado inicial (S) para o estado objetivo (G)</b> andando apenas sobre peças congeladas (F) e evitando buracos (H). No entanto, o gelo é escorregadio, **então você nem sempre se moverá na direção pretendida (ambiente estocástico)**

## Pré-requisitos 🏗️

Antes de mergulhar no notebook **você precisa entender**:
- Os fundamentos da aprendizagem por reforço (MC, TD, hipótese de recompensas...)
- Q-aprendizagem

## Etapa 1: Instalando as dependências no Google Colab

In [1]:
!pip install gymnasium
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 21.5 MB/s eta 0:00:00


## Etapa 0: Importando as dependências 📚

Usamos 3 bibliotecas:

- `Numpy` para nosso Qtable
- `Gymnasium` para nosso ambiente FrozenLake
- `Random` para gerar números aleatórios

In [2]:
import numpy as np
import gymnasium as gym
import random

## Passo 1: Criando o ambiente 🎮

- Aqui criaremos o ambiente FrozenLake 8x8.
- Gymnasium é uma biblioteca <b> composta por diversos ambientes que podemos usar para treinar nossos agentes.</b>
- No nosso caso optamos por usar Frozen Lake.

In [3]:
env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=True, render_mode="human")

## Etapa 2: Criando a tabela Q e inicializando-a 🗄️

- Agora, vamos criar nossa Q-table, para saber de quantas linhas (estados) e colunas (ações) precisamos, precisamos calcular o action_size e o state_size
- Gymnasium nos fornece uma maneira de fazer isso: `env.action_space.n` e `env.observation_space.n`

In [4]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [5]:
# Create our Q table with state_size rows and action_size columns (64x4)
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Etapa 3: Criando os hiperparâmetros ⚙️

- Aqui, especificaremos os hiperparâmetros

In [6]:
total_episodes = 500       # Total episodes
learning_rate = 0.7          # Learning rate
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.005            # Exponential decay rate for exploration prob


![texto alternativo](https://miro.medium.com/v2/resize:fit:1400/1*tyIE_430xbBRzrrjUdYLQw.png)


## Etapa 4: O algoritmo de aprendizagem Q 🧠

- Agora implementamos o algoritmo de aprendizagem Q:


In [7]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state, _ = env.reset()
    state = int(state)
    step = 0
    done = False
    total_rewards = 0

    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)

        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
            #print(exp_exp_tradeoff, "action", action)

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
            #print("action random", action)


        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])

        total_rewards += reward

        # Our new state is state
        state = new_state

        # If done (if we're dead) : finish episode
        if done:
            break

    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    rewards.append(total_rewards)


print ("Pontuação no tempo: " +  str(sum(rewards)/total_episodes))
print(qtable)

Pontuação no tempo: 0.066
[[1.42206755e-01 1.42749721e-01 1.47819383e-01 8.79129329e-02]
 [3.91908243e-02 1.91711770e-02 2.10721563e-02 5.78597804e-02]
 [4.24972861e-02 3.82009692e-02 4.05331992e-02 3.95196777e-02]
 [3.07498333e-02 4.45504972e-03 8.14724307e-03 3.69286210e-02]
 [1.71734300e-01 4.37909887e-02 1.47757856e-01 1.10047758e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.28984014e-02 4.00015322e-03 4.73139915e-04 2.55574906e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.34688028e-02 7.86954259e-02 1.23756821e-01 2.85913980e-01]
 [1.97841165e-01 3.87385114e-01 2.94348589e-02 1.58841010e-01]
 [2.57098521e-01 2.20074686e-02 6.65491122e-02 6.99023811e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.21654153e-01 0.00000000e+00 6.91746917e-01 0.00000000e+00]
 [3.96665525e-01 4.91705894e-01 7.76507620e-01 3.93889334e-01]
 [0.00000000e+00 0.00000000e+

## Etapa 5: Usando nossa tabela Q para jogar FrozenLake! 👾

- Após 10.000 episódios, nossa tabela Q pode ser usada como uma "folha de dicas" para jogar FrozenLake"
- Ao executar este celular você poderá ver nosso agente jogando FrozenLake.

In [8]:
for episode in range(5):
    state, info = env.reset()
    state = int(state)
    step = 0
    done = False
    max_steps = 100
    print("****************************************************")
    print("EPISODIO ", episode)

    for step in range(max_steps):

        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])

        new_state, reward, terminated, truncated, _ = env.step(action)

        done = terminated or truncated

        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            if new_state == 15:
                print("Chegamos no Objetivo 🏆")
            else:
                print("Caímos em um Obstáculo ☠️")

            # We print the number of step it took.
            print("Número de passos", step)

            break
        state = int(new_state)
env.close()

****************************************************
EPISODIO  0
Caímos em um Obstáculo ☠️
Número de passos 42
****************************************************
EPISODIO  1
Chegamos no Objetivo 🏆
Número de passos 47
****************************************************
EPISODIO  2
Chegamos no Objetivo 🏆
Número de passos 45
****************************************************
EPISODIO  3
Caímos em um Obstáculo ☠️
Número de passos 4
****************************************************
EPISODIO  4
Chegamos no Objetivo 🏆
Número de passos 20
